In [2]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import re
import collections 
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Data gathering and data cleaning

In [3]:
data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/train.csv')
test_data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/test.csv')

In [4]:
def get_clean_data(A):
    BB = A.strip().split("\n")
    B = []
    for line in BB:
        B.extend(line.strip().split(" "))
    handles  = []
    hashtags = []
    for item in B:
        i = 0
        while (i < len(item)):
            if item[i]=='@':
                handle = item[i+1:] 
                #handle = re.sub('[^a-zA-z0-9\s]','',handle)
                #handle = handle.lower()
                handles.append(handle)
                B.remove(item)
                break
            elif item[i]=='#':
                hashtag = item[i+1:] 
                #hashtag = re.sub('[^a-zA-z0-9\s]','',hashtag)
                #hashtag = hashtag.lower()
                hashtags.append(hashtag)
                B.remove(item)
                break
            else: 
                i += 1 
    text = " ".join(B)
    #text = re.sub('[^a-zA-z0-9\s]','',text)
    #text = text.lower()
    return handles, hashtags, text

In [5]:
def string_from_list(A):
    if len(A)>0:
        B = (" ".join(A)).lower()
        return re.sub('[^a-zA-z0-9\s]','',B)
    else:
        return None 

In [6]:
data['handle']    = data['handle'].replace({'realDonaldTrump':1, 'HillaryClinton':0})

data['tweet']     = data['tweet'].apply((lambda x: re.sub(r"http\S+", 'urlurlurlurl', x)))

data['mentioned'] = data['tweet'].apply((lambda x: get_clean_data(x)[0]))
data['mentioned'] = data['mentioned'].apply((lambda x: string_from_list(x)))

data['hashtags']  = data['tweet'].apply((lambda x: get_clean_data(x)[1]))
data['hashtags']  = data['hashtags'].apply((lambda x: string_from_list(x) ))

data['tweet']     = data['tweet'].apply(lambda x: x.lower())
data['tweet']     = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['tweet']     = data['tweet'].apply((lambda x: re.sub(' +',' ',x)))
data['tweet']     = data['tweet'].apply((lambda x: re.sub(r'(\n+)(?=[A-Z])', r' ', x)))


test_data['tweet']     = test_data['tweet'].apply((lambda x: re.sub(r"http\S+", 'urlurlurlurl', x)))

test_data['mentioned'] = test_data['tweet'].apply((lambda x: get_clean_data(x)[0]))
test_data['mentioned'] = test_data['mentioned'].apply((lambda x: string_from_list(x) ))

test_data['hashtags']  = test_data['tweet'].apply((lambda x: get_clean_data(x)[1]))
test_data['hashtags']  = test_data['hashtags'].apply((lambda x: string_from_list(x) ))

test_data['tweet']     = test_data['tweet'].apply(lambda x: x.lower())
test_data['tweet']     = test_data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
test_data['tweet']     = test_data['tweet'].apply((lambda x: re.sub(' +',' ',x)))
test_data['tweet']     = test_data['tweet'].apply((lambda x: re.sub(r'(\n+)(?=[A-Z])', r' ', x)))

all_tweets = pd.concat([data['tweet'], test_data['tweet']])

X       = data['tweet']
Y       = np.array(data['handle'].tolist())
X_test  = test_data['tweet']

In [7]:
data_hashtag             = data.drop(['tweet', 'mentioned'], axis=1)
data_hashtag_clean       = data_hashtag.dropna() 

X_hashtag_bow_pd         = data_hashtag_clean['hashtags']
Y_hashtag_bow_pd         = data_hashtag_clean['handle']

X_train_hashtag_bow_pd, X_val_hashtag_bow_pd, Y_train_hashtag_bow_pd, Y_val_hashtag_bow_pd = \
train_test_split(X_hashtag_bow_pd, Y_hashtag_bow_pd,  test_size=0.1)

data_hashtag_test        = test_data.drop(['tweet', 'mentioned'], axis=1)
data_hashtag_clean_test  = data_hashtag_test.dropna() 
X_test_hashtag_bow_pd    = data_hashtag_clean_test['hashtags']

In [8]:
data_mentioned             = data.drop(['tweet', 'hashtags'], axis=1)
data_mentioned_clean       = data_mentioned.dropna() 

X_mentioned_bow_pd         = data_mentioned_clean['mentioned']
Y_mentioned_bow_pd         = data_mentioned_clean['handle']

X_train_mentioned_bow_pd, X_val_mentioned_bow_pd, Y_train_mentioned_bow_pd, Y_val_mentioned_bow_pd = \
train_test_split(X_mentioned_bow_pd, Y_mentioned_bow_pd,  test_size=0.1)

data_mentioned_test        = test_data.drop(['tweet', 'hashtags'], axis=1)
data_mentioned_clean_test  = data_mentioned_test.dropna() 
X_test_mentioned_bow_pd    = data_mentioned_clean_test['mentioned']

In [9]:
print(X_hashtag_bow_pd.index[5])
print(X_hashtag_bow_pd.head(7), "\n\n")
print(data.iloc[[X_hashtag_bow_pd.index[5]]])

11
0               makeamericagreatagain
4                       newyorkvalues
7                        decision2016
8                              diwali
10    trump2016 makeamericagreatagain
11                               lesm
12                          womancard
Name: hashtags, dtype: object 


    handle                                              tweet mentioned  \
11       1  thank you to our law enforcement officers lesm...      None   

   hashtags  
11     lesm  


# Training bag of words models

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [10]:
count_vec_hashtag         = CountVectorizer()
tfidf_transformer_hashtag = TfidfTransformer()

X_train_counts_hashtag    = count_vec_hashtag.fit_transform(X_train_hashtag_bow_pd)
X_train_tfidf_hashtag     = tfidf_transformer_hashtag.fit_transform(X_train_counts_hashtag)

X_val_counts_hashtag      = count_vec_hashtag.transform(X_val_hashtag_bow_pd)
X_val_tfidf_hashtag       = tfidf_transformer_hashtag.transform(X_val_counts_hashtag)

In [38]:
hyperVec                  = [1e6,1e5,1e4,1e3,1e2,1e1,1e0,1e-1,1e-2]
loss_dic_bow_hashtag      = {}

for C in hyperVec:
    lf_hashtag_tmp = LogisticRegression(C=C)
    lf_hashtag_tmp.fit(X_train_tfidf_hashtag, Y_train_hashtag_bow_pd)
    y_val_predicted_hashtag_tmp = lf_hashtag_tmp.predict(X_val_tfidf_hashtag)
    
    loss_dic_bow_hashtag[C] = log_loss(y_pred = y_val_predicted_hashtag_tmp, y_true =  Y_val_hashtag_bow_pd)

    
C_optimal_hashtag = 1
#min(loss_dic_bow_hashtag, key=loss_dic_bow_hashtag.get)
#loss_dic_bow_hashtag

In [40]:
X_counts_hashtag          = count_vec_hashtag.fit_transform(X_hashtag_bow_pd)
X_tfidf_hashtag           = tfidf_transformer_hashtag.fit_transform(X_counts_hashtag)

X_test_hashtag_counts     = count_vec_hashtag.transform(X_test_hashtag_bow_pd)
X_test_hashtag_tfidf      = tfidf_transformer_hashtag.transform(X_test_hashtag_counts)

lf_hashtag = LogisticRegression(C=C_optimal_hashtag)
lf_hashtag.fit(X_tfidf_hashtag, Y_hashtag_bow_pd)

Y_hashtag_bow_predicted_test = lf_hashtag.predict(X_test_hashtag_tfidf)

#print(Y_hashtag_bow_predicted_test)

In [41]:
count_vec_mentioned          = CountVectorizer()
tfidf_transformer_mentioned  = TfidfTransformer()

X_train_counts_mentioned    = count_vec_mentioned.fit_transform(X_train_mentioned_bow_pd)
X_train_tfidf_mentioned     = tfidf_transformer_mentioned.fit_transform(X_train_counts_mentioned)

X_val_counts_mentioned      = count_vec_mentioned.transform(X_val_mentioned_bow_pd)
X_val_tfidf_mentioned       = tfidf_transformer_mentioned.transform(X_val_counts_mentioned)

In [43]:
hyperVec                  = [1e6,1e5,1e4,1e3,1e2,1e1,1e0,1e-1,1e-2]
loss_dic_bow_mentioned      = {}

for C in hyperVec:
    lf_mentioned_tmp = LogisticRegression(C=C)
    lf_mentioned_tmp.fit(X_train_tfidf_mentioned, Y_train_mentioned_bow_pd)
    y_val_predicted_mentioned_tmp = lf_mentioned_tmp.predict(X_val_tfidf_mentioned)
    
    loss_dic_bow_mentioned[C] = log_loss(y_pred = y_val_predicted_mentioned_tmp, y_true =  Y_val_mentioned_bow_pd)

    
C_optimal_mentioned = 5
#min(loss_dic_bow_mentioned, key=loss_dic_bow_mentioned.get)
#loss_dic_bow_mentioned

In [45]:
X_counts_mentioned           = count_vec_mentioned.fit_transform(X_mentioned_bow_pd)
X_tfidf_mentioned            = tfidf_transformer_mentioned.fit_transform(X_counts_mentioned)

X_test_mentioned_counts      = count_vec_mentioned.transform(X_test_mentioned_bow_pd)
X_test_mentioned_tfidf       = tfidf_transformer_mentioned.transform(X_test_mentioned_counts)

lf_mentioned = LogisticRegression(C=C_optimal_mentioned)
lf_mentioned.fit(X_tfidf_mentioned, Y_mentioned_bow_pd)

Y_mentioned_bow_predicted_test = lf_mentioned.predict(X_test_mentioned_tfidf)

#print(Y_mentioned_bow_predicted_test)

In [51]:
X_test_mentioned_tfidf 

<447x792 sparse matrix of type '<class 'numpy.float64'>'
	with 405 stored elements in Compressed Sparse Row format>

In [ ]:
X_all = []
for i in range(X.shape[0]):
    X_all.append(X[i].split())
    
for i in range(X_test.shape[0]):
    X_all.append(X_test[i].split())

In [ ]:
tweets = all_tweets.tolist()
words  = " ".join(tweets)
words  = " ".join(words.split("\n")) 
words  = " ".join(words.split("\t"))
words  = " ".join(words.split("\xa0")).split()
whole_text = " ".join(words)

count          = collections.Counter(words)
count_clean    = [(item, count[item]) for item in count if count[item]>1]
vocab_size     = len(count_clean)
num_words      = vocab_size+2 

dic = {}

for i in range(vocab_size):
    dic[count_clean[i][0]] = (i+1, count_clean[i][1])

In [ ]:
def string_to_token_list(A):
    B      = A.strip().split(" ")
    output = []
    for item in B:
        if item in dic:
            output.append(dic[item][0])
        else:
            output.append(vocab_size + 1)
    return output

In [ ]:
X_tokens       = X.apply(lambda x: string_to_token_list(x))
X_test_tokens  = X_test.apply(lambda x: string_to_token_list(x)) 

num_tokens = [len(tokens) for tokens in X_tokens]
num_tokens = np.array(num_tokens)
max_tokens = int(np.max(num_tokens))

X_pad       = []
X_test_pad  = []

for item in X_tokens:
    n = len(item)
    X_pad.append([0]*(max_tokens-n) + item)

for item in X_test_tokens:
    n = len(item)
    X_test_pad.append([0]*(max_tokens-n) + item)
    
X_pad        = np.array(X_pad)
X_test_pad   = np.array(X_test_pad)
X_all_pad    = np.concatenate((X_pad , X_test_pad), axis=0)

In [ ]:
X_train_pad, X_val_pad, Y_train_pad, Y_val_pad = train_test_split(X_pad, Y, test_size = 0.1, random_state = 42)

# Hyper parameter selection

In [ ]:
lr                  = 1e-4
batch_size          = 64
embedding_size      = 128
lstm_out            = 32
lstm_dropout_prob   = 0.5
lstm_forget_bias    = 1.0
hidden_layer_size_1 = 32
hidden_layer_size_2 = 16

# Word embedding initialization 

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(X_all, iter=10, min_count=2, size=embedding_size, workers=4)
#words = list(model.wv.vocab)
#print(words)
#print(model.wv.similar_by_word('makeamericagreatagain'))
#model.wv['trump'] 

In [ ]:
embedding_matrix = np.zeros((1,embedding_size), np.float32)

for i in range(vocab_size):
    word             = count_clean[i][0]
    word_vec         = model.wv[word]
    embedding_matrix = np.vstack((embedding_matrix, word_vec))

embedding_matrix = np.vstack((embedding_matrix, 5* np.ones((1,embedding_size), np.float32)))

# Utility functions

In [ ]:
def evaluate_test_data_set(filename, X_test_pad, sess):
        test_size   = X_test_pad.shape[0]
        test_remain = batch_size - (test_size % batch_size) 
        X_test_pad  = np.concatenate((X_test_pad, np.zeros((test_remain, max_tokens), np.float32)), axis=0)
        m = test_size // batch_size 
        pred_test_vals = np.empty((0, 2), np.float32)

        for i in range(m+1):
            input_test_batch = X_test_pad[i * batch_size : (i+1) * batch_size, :]
            test_preds_list  = sess.run([predictions], feed_dict = {X : input_test_batch})
            pred_test_batch  = np.asarray(test_preds_list).reshape(batch_size, 2)
            pred_test_vals   = np.concatenate((pred_test_vals, pred_test_batch), axis=0)
                 
            
        with open(filename,"w+") as outputfile:
            outputfile.write("id,realDonaldTrump,HillaryClinton\n")
            for j in range(test_size):
                hillary = pred_test_vals[j][1]
                donald  = pred_test_vals[j][0]
                outputfile.write(str(j)+","+str(donald)+","+str(hillary)+"\n")

In [ ]:
def get_batch(X, Y, size):
    rdm      = np.random.choice(X.shape[0], size , replace = False)
    y1       = Y[rdm].reshape((size,1))
    y2       = (y1+1)%2
    Y_out    = np.concatenate((y1, y2), axis=1)
    X_out    = X[rdm,:]
    return X_out, Y_out

#### Look at 
https://github.com/bernhard2202/twitter-sentiment-analysis/blob/master/model/lstm.py

For a similar implementation

# Computational graph

In [ ]:
with tf.device("/cpu:0"):
    
    tf.reset_default_graph()
    tf.set_random_seed(0)

    X = tf.placeholder(tf.int32, shape=[batch_size, max_tokens])
    Y = tf.placeholder(tf.float32, shape=[batch_size,2])
        
    embeddings     = tf.Variable(embedding_matrix)    
    embed          = tf.nn.embedding_lookup(embeddings, X)
    embed          = tf.unstack(embed, max_tokens, 1)
    
    lstm_cell                 = tf.contrib.rnn.BasicLSTMCell(lstm_out, forget_bias = lstm_forget_bias)
    lstm_cell                 = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=lstm_dropout_prob)
    lstm_output , lstm_state  = tf.contrib.rnn.static_rnn(lstm_cell, inputs=embed , dtype=tf.float32)
    
    outputs                   = tf.stack(lstm_output)
    outputs                   = tf.gather(outputs, max_tokens-1, axis=0)
    outputs                   = tf.reshape(outputs, [batch_size, lstm_out])
    lstm_final_output         = tf.reshape(outputs, [batch_size , lstm_out])
    hidden_layer_1_output     = tf.layers.dense(lstm_final_output, hidden_layer_size_1)
    hidden_layer_2_output     = tf.layers.dense(hidden_layer_1_output, hidden_layer_size_2)
    
    out_weight      = tf.Variable(tf.random_normal([hidden_layer_size_2, 2]))
    out_bias        = tf.Variable(tf.random_normal([2]))
    
    scores          = tf.nn.xw_plus_b(hidden_layer_2_output, out_weight,out_bias)
    predictions     = tf.nn.softmax(scores)
    
    losses          = tf.nn.softmax_cross_entropy_with_logits_v2(logits = scores,
                                                              labels = Y)
    loss            = tf.reduce_mean(losses)
    
    correct_pred    = tf.equal(tf.argmax(predictions, 1), tf.argmax(Y, 1))
    accuracy        = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    optimizer       = tf.train.AdamOptimizer(lr).minimize(loss)

# Running the computational graph

In [ ]:
epochs           = 1500
display_epoch    = 100

loss_val_vec     = []
accur_val_vec    = []
loss_train_vec   = []
accur_train_vec  = []
epoch_vec = []

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(epochs):
            batch_inputs, batch_labels = get_batch(X_train_pad, Y_train_pad, batch_size)
            loss_train, accur_train, _ = sess.run([loss, accuracy, optimizer],
                                                  feed_dict = {X : batch_inputs,
                                                               Y : batch_labels})
            
            batch_inputs, batch_labels = get_batch(X_val_pad, Y_val_pad, batch_size)
            loss_val, accur_val = sess.run([loss, accuracy],
                                           feed_dict = {X : batch_inputs,
                                                        Y : batch_labels})
                
            loss_train_vec.append(loss_train)
            accur_train_vec.append(accur_train)
            loss_val_vec.append(loss_val)
            accur_val_vec.append(accur_val)
            epoch_vec.append(epoch)
            
            if epoch % display_epoch == 0:
                print("Epoch " +str(epoch) + ", Batch loss = ", 
                      "%.3f, " % loss_train , "training accuray = ",  "%.3f, " % accur_train, "val loss = ",
                      "%.3f, " % loss_val , "val accuray = ",  "%.3f, " % accur_val)
        
        evaluate_test_data_set("/Users/mohsenkiskani/Downloads/submissions/submission_lstm_results.csv",
                               X_test_pad, sess)

# Plotting the results

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig1, (ay1, ay2) = plt.subplots(2, 1)
ay1.plot(epoch_vec, loss_train_vec)
ay2.plot(epoch_vec, loss_val_vec)

fig2, (ay3, ay4) = plt.subplots(2, 1)
ay3.plot(epoch_vec, accur_train_vec)
ay4.plot(epoch_vec, accur_val_vec)

ay1.set_title('Training batch loss vs epoch')
ay1.set_xlabel('Epoch')
ay1.set_ylabel('Training batch loss')

ay2.set_title('Validation batch loss vs epoch')
ay2.set_xlabel('Epoch')
ay2.set_ylabel('Validation batch loss')

ay3.set_title('Training batch accuracy vs epoch')
ay3.set_xlabel('Epoch')
ay3.set_ylabel('Train accuracy')

ay4.set_title('Validation batch accuracy vs epoch')
ay4.set_xlabel('Epoch')
ay4.set_ylabel('Validation accuracy')

fig1.set_size_inches(18.5, 10.5)
fig1.savefig('loss.png', dpi=100)

fig2.set_size_inches(18.5, 10.5)
fig2.savefig('loss.png', dpi=100)
plt.show()